In [9]:
import pandas as pd
import os

In [14]:
df = pd.read_csv('CATANDUANES_case_information.csv')
df.head()

,Unnamed: 0,CaseCode,Age,AgeGroup,Sex,DateSpecimen,DateResultRelease,DateRepConf,DateDied,DateRecover,...,ProvRes,CityMunRes,CityMuniPSGC,BarangayRes,BarangayPSGC,HealthStatus,Quarantined,DateOnset,Pregnanttab,ValidationStatus
0,19761,C781369,50.0,50 to 54,MALE,2020-05-30,NaN,2020-06-06,NaN,2020-07-07,...,CATANDUANES,SAN MIGUEL,PH052009000,NaN,NaN,RECOVERED,YES,NaN,NaN,Age or Birthdate is Invalid\r\nCase has Admitt...
1,29313,C636837,47.0,45 to 49,MALE,NaN,NaN,2020-06-22,NaN,2020-07-08,...,CATANDUANES,BAGAMANOC,PH052001000,NaN,NaN,RECOVERED,YES,NaN,NaN,Age or Birthdate is Invalid
2,43311,C951557,1.0,0 to 4,MALE,2020-07-02,2020-07-04,2020-07-06,NaN,2020-07-23,...,CATANDUANES,VIRAC (CAPITAL),PH052011000,NaN,NaN,RECOVERED,NO,2020-06-28,NaN,Age or Birthdate is Invalid
3,45859,C639154,21.0,20 to 24,MALE,2020-06-30,2020-07-05,2020-07-08,NaN,NaN,...,CATANDUANES,CARAMORAN,PH052004000,NaN,NaN,RECOVERED,NO,2020-06-30,NaN,"Removal Type is ""Recovered"", but no Recovered ..."
4,49871,C619176,22.0,20 to 24,FEMALE,2020-07-06,2020-07-07,2020-07-10,NaN,2020-07-23,...,CATANDUANES,VIRAC (CAPITAL),PH052011000,NaN,NaN,RECOVERED,YES,NaN,NO,Age or Birthdate is Invalid


In [7]:
df.columns

Index(['Unnamed: 0', 'CaseCode', 'Age', 'AgeGroup', 'Sex', 'DateSpecimen',
       'DateResultRelease', 'DateRepConf', 'DateDied', 'DateRecover',
       'RemovalType', 'DateRepRem', 'Admitted', 'RegionRes', 'ProvRes',
       'CityMunRes', 'CityMuniPSGC', 'BarangayRes', 'BarangayPSGC',
       'HealthStatus', 'Quarantined', 'DateOnset', 'Pregnanttab',
       'ValidationStatus'],
      dtype='object')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4258 entries, 0 to 4257
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         4258 non-null   int64  
 1   CaseCode           4258 non-null   object 
 2   Age                4249 non-null   float64
 3   AgeGroup           4249 non-null   object 
 4   Sex                4258 non-null   object 
 5   DateSpecimen       3610 non-null   object 
 6   DateResultRelease  3603 non-null   object 
 7   DateRepConf        4258 non-null   object 
 8   DateDied           39 non-null     object 
 9   DateRecover        825 non-null    object 
 10  RemovalType        4256 non-null   object 
 11  DateRepRem         4256 non-null   object 
 12  Admitted           3968 non-null   object 
 13  RegionRes          4258 non-null   object 
 14  ProvRes            4258 non-null   object 
 15  CityMunRes         4055 non-null   object 
 16  CityMuniPSGC       3999 

In [12]:
df.drop(['Unnamed: 0', 'CaseCode', 'RegionRes', 'ProvRes', 'DateSpecimen', 'DateResultRelease', 'DateDied', 'DateRecover', 'DateRepRem',
'CityMuniPSGC', 'BarangayRes', 'BarangayPSGC'], axis=1, inplace=True)

In [14]:
df.head()

,Age,AgeGroup,Sex,DateRepConf,RemovalType,Admitted,CityMunRes,HealthStatus,Quarantined,DateOnset,Pregnanttab,ValidationStatus
0,50.0,50 to 54,MALE,2020-06-06,RECOVERED,YES,SAN MIGUEL,RECOVERED,YES,NaN,NaN,Age or Birthdate is Invalid\r\nCase has Admitt...
1,47.0,45 to 49,MALE,2020-06-22,RECOVERED,NO,BAGAMANOC,RECOVERED,YES,NaN,NaN,Age or Birthdate is Invalid
2,1.0,0 to 4,MALE,2020-07-06,RECOVERED,YES,VIRAC (CAPITAL),RECOVERED,NO,2020-06-28,NaN,Age or Birthdate is Invalid
3,21.0,20 to 24,MALE,2020-07-08,RECOVERED,NO,CARAMORAN,RECOVERED,NO,2020-06-30,NaN,"Removal Type is ""Recovered"", but no Recovered ..."
4,22.0,20 to 24,FEMALE,2020-07-10,RECOVERED,NO,VIRAC (CAPITAL),RECOVERED,YES,NaN,NO,Age or Birthdate is Invalid


In [15]:
df['Pregnanttab'].unique()

array([nan, 'NO'], dtype=object)

In [16]:
df['Pregnanttab'] = df['Pregnanttab'].fillna('YES')

Can probably include as covariate yung other columns.

## Create new datasets that align with the research papers' processed datasets

### Create Year feature

In [21]:
# we're just interested in getting the year so we just parse it from the string
def get_year(date):
    if '/' in date:
        dt = date.split('/')
        return (dt[0] if len(dt[0]) == 4 else (dt[1] if len(dt[1]) == 4 else dt[2]))
    else:
        dt = date.split('-')
        return (dt[0] if len(dt[0]) == 4 else (dt[1] if len(dt[1]) == 4 else dt[2]))
df['Year'] = df['DateRepConf'].apply(get_year)

In [22]:
df['Year'].unique()

array(['2020', '2021', '2022', '2023'], dtype=object)

For `pob_abs`, we aggregate the dataset based on year, towm/municipality, sex, and age since we don't have access to hospitalization and vaccination.

In [23]:
pob_towns = df.copy()

In [26]:
pob_towns = pob_towns.groupby(['Year', 'CityMunRes', 'Sex', 'AgeGroup']).size().reset_index(name='count')

In [27]:
pob_towns

,Year,CityMunRes,Sex,AgeGroup,count
0,2020,BAGAMANOC,MALE,20 to 24,3
1,2020,BAGAMANOC,MALE,25 to 29,1
2,2020,BAGAMANOC,MALE,45 to 49,1
3,2020,BARAS,FEMALE,15 to 19,1
4,2020,BARAS,FEMALE,20 to 24,1
...,...,...,...,...,...
811,2023,VIRAC (CAPITAL),MALE,5 to 9,1
812,2023,VIRAC (CAPITAL),MALE,55 to 59,1
813,2023,VIRAC (CAPITAL),MALE,65 to 69,2
814,2023,VIRAC (CAPITAL),MALE,70 to 74,4


In [28]:
pob_towns.to_csv('./Data - Catanduanes/1_Processed/pob_towns.csv')

We now work on the aggregation for `dat_cas`.

In [30]:
dat_cas = df.copy()

We fix `DateRepConf` orientation.

In [33]:
def fixDatetimeOrientation(date):
    if '/' in date:
        dt = date.split('/')
        return f'{dt[2]}-{dt[0]}-{dt[1]}'
    return date

dat_cas['DateRepConf'] = dat_cas['DateRepConf'].apply(fixDatetimeOrientation)
dat_cas['DateRepConf'] = pd.to_datetime(dat_cas['DateRepConf'])

In [35]:
dat_cas = dat_cas.groupby(['DateRepConf', 'CityMunRes', 'Sex']).size().reset_index(name='count')
dat_cas.head()

,DateRepConf,CityMunRes,Sex,count
0,2020-06-06,SAN MIGUEL,MALE,1
1,2020-06-22,BAGAMANOC,MALE,1
2,2020-07-06,VIRAC (CAPITAL),MALE,1
3,2020-07-08,CARAMORAN,MALE,1
4,2020-07-10,VIRAC (CAPITAL),FEMALE,2


In [36]:
dat_cas.to_csv('./Data - Catanduanes/1_Processed/dat_cas.csv')

We now try to recreate `dat_edat`.

In [38]:
dat_age = df.copy()

dat_age.groupby(['DateRepConf', 'Sex', 'AgeGroup']).size().reset_index(name='count')

dat_age.to_csv('./Data - Catanduanes/1_Processed/dat_age.csv')

We recreate `Tdat_cas`.

In [39]:
total_cases_per_date = df.copy()

total_cases_per_date = df['DateRepConf'].value_counts()

total_cases_per_date.to_csv('./Data - Catanduanes/1_Processed/total_cases_per_date.csv')

We recreate `dat_covar`.

In [48]:
covariates = df.copy()

covariates = covariates.groupby('CityMunRes').agg(
    N = ('CityMunRes', 'count'),
    N_women=('Sex', lambda x: (x == 'FEMALE').sum()),
    N_70=('Age', lambda x: (x > 70).sum())
).reset_index()

In [51]:
def perc70Cat(perc):
    if perc < 12:
        return '<12'
    elif 14-perc <= 0.000000000000000000000001:
        return '12-14'
    elif 16-perc <= 0.000000000000000000000001:
        return '14-16'
    return '>16'

covariates['perc_women'] = (covariates['N_women']/covariates['N'])*100
covariates['perc_70'] = (covariates['N_70']/covariates['N'])*100
covariates['perc_70_cat'] = covariates['perc_70'].apply(perc70Cat)
covariates.head()

,CityMunRes,N,N_women,N_70,perc_women,perc_70,perc_70_cat
0,BAGAMANOC,212,103,19,48.584906,8.962264,<12
1,BARAS,173,80,20,46.242775,11.560694,<12
2,BATO,326,182,36,55.828221,11.042945,<12
3,CARAMORAN,358,195,30,54.469274,8.379888,<12
4,GIGMOTO,150,86,17,57.333333,11.333333,<12


In [52]:
covariates.to_csv('./Data - Catanduanes/1_Processed/covariates.csv')

In [2]:
pt = pd.read_csv('./Data - Catanduanes/1_Processed/pob_towns.csv')
pt.head()

,Unnamed: 0,Year,CityMunRes,Sex,AgeGroup,count
0,0,2020,BAGAMANOC,MALE,20 to 24,3
1,1,2020,BAGAMANOC,MALE,25 to 29,1
2,2,2020,BAGAMANOC,MALE,45 to 49,1
3,3,2020,BARAS,FEMALE,15 to 19,1
4,4,2020,BARAS,FEMALE,20 to 24,1


In [3]:
df = pd.read_csv('CATANDUANES_case_information.csv')
df.head()

,Unnamed: 0,CaseCode,Age,AgeGroup,Sex,DateSpecimen,DateResultRelease,DateRepConf,DateDied,DateRecover,...,ProvRes,CityMunRes,CityMuniPSGC,BarangayRes,BarangayPSGC,HealthStatus,Quarantined,DateOnset,Pregnanttab,ValidationStatus
0,19761,C781369,50.0,50 to 54,MALE,2020-05-30,NaN,2020-06-06,NaN,2020-07-07,...,CATANDUANES,SAN MIGUEL,PH052009000,NaN,NaN,RECOVERED,YES,NaN,NaN,Age or Birthdate is Invalid\r\nCase has Admitt...
1,29313,C636837,47.0,45 to 49,MALE,NaN,NaN,2020-06-22,NaN,2020-07-08,...,CATANDUANES,BAGAMANOC,PH052001000,NaN,NaN,RECOVERED,YES,NaN,NaN,Age or Birthdate is Invalid
2,43311,C951557,1.0,0 to 4,MALE,2020-07-02,2020-07-04,2020-07-06,NaN,2020-07-23,...,CATANDUANES,VIRAC (CAPITAL),PH052011000,NaN,NaN,RECOVERED,NO,2020-06-28,NaN,Age or Birthdate is Invalid
3,45859,C639154,21.0,20 to 24,MALE,2020-06-30,2020-07-05,2020-07-08,NaN,NaN,...,CATANDUANES,CARAMORAN,PH052004000,NaN,NaN,RECOVERED,NO,2020-06-30,NaN,"Removal Type is ""Recovered"", but no Recovered ..."
4,49871,C619176,22.0,20 to 24,FEMALE,2020-07-06,2020-07-07,2020-07-10,NaN,2020-07-23,...,CATANDUANES,VIRAC (CAPITAL),PH052011000,NaN,NaN,RECOVERED,YES,NaN,NO,Age or Birthdate is Invalid


In [4]:
# we're just interested in getting the year so we just parse it from the string
def get_year(date):
    if '/' in date:
        dt = date.split('/')
        return (dt[0] if len(dt[0]) == 4 else (dt[1] if len(dt[1]) == 4 else dt[2]))
    else:
        dt = date.split('-')
        return (dt[0] if len(dt[0]) == 4 else (dt[1] if len(dt[1]) == 4 else dt[2]))
df['Year'] = df['DateRepConf'].apply(get_year)

In [5]:
pob_towns = df.copy()

In [17]:
def vac_func(age):
    if age >= 0 and age <= 14:
        return "0 to 14"
    elif age >= 15 and age <= 44:
        return "15 to 44"
    elif age >= 45 and age <= 59:
        return "45 to 59"
    elif age >= 60 and age <= 69:
        return "60 to 69"
    elif age >= 70 and age <= 79:
        return "70 to 79"
    return "80+"

def hosp_func(age):
    if age >= 0 and age <= 11:
        return "0 to 11"
    elif age >= 12 and age <= 14:
        return "12 to 14"
    if age >= 15 and age <= 19:
        return "15 to 19"
    elif age >= 20 and age <= 24:
        return "20 to 24"
    elif age >= 25 and age <= 29:
        return "25 to 29"
    elif age >= 30 and age <= 34:
        return "30 to 34"
    elif age >= 35 and age <= 39:
        return "35 to 39"
    elif age >= 40 and age <= 44:
        return "40 to 44"
    elif age >= 45 and age <= 49:
        return "45 to 49"
    elif age >= 50 and age <= 54:
        return "50 to 54"
    elif age >= 55 and age <= 59:
        return "55 to 59"
    elif age >= 60 and age <= 64:
        return "60 to 64"
    elif age >= 65 and age <= 69:
        return "65 to 69"
    elif age >= 70 and age <= 74:
        return "70 to 74"
    elif age >= 75 and age <= 79:
        return "75 to 79"
    return "80+"


In [ ]:
pob_towns = pob_towns.groupby(['Year', 'CityMunRes', 'Sex', 'Age', 'AgeGroup']).size().reset_index(name='count')
pob_towns['AgeGroupVac'] = pob_towns['Age'].apply(vac_func)
pob_towns['AgeGroupHosp'] = pob_towns['Age'].apply(hosp_func)

In [7]:
pob_towns.head()

,Year,CityMunRes,Sex,Age,AgeGroup,count,AgeGroupVac,AgeGroupHosp
0,2020,BAGAMANOC,MALE,22.0,20 to 24,1,15 to 44,20 to 24
1,2020,BAGAMANOC,MALE,23.0,20 to 24,1,15 to 44,20 to 24
2,2020,BAGAMANOC,MALE,24.0,20 to 24,1,15 to 44,20 to 24
3,2020,BAGAMANOC,MALE,26.0,25 to 29,1,15 to 44,25 to 29
4,2020,BAGAMANOC,MALE,47.0,45 to 49,1,45 to 59,45 to 49


In [8]:
pob_towns.to_csv('./Data - Catanduanes/1_Processed/pob_towns.csv')

In [11]:
pob_towns = pd.read_csv('./Data - Catanduanes/1_Processed/pob_towns.csv')

In [12]:
pob_towns.head()

,Unnamed: 0,Year,CityMunRes,Sex,Age,AgeGroup,count,AgeGroupVac,AgeGroupHosp
0,0,2020,BAGAMANOC,MALE,22.0,20 to 24,1,15 to 44,20 to 24
1,1,2020,BAGAMANOC,MALE,23.0,20 to 24,1,15 to 44,20 to 24
2,2,2020,BAGAMANOC,MALE,24.0,20 to 24,1,15 to 44,20 to 24
3,3,2020,BAGAMANOC,MALE,26.0,25 to 29,1,15 to 44,25 to 29
4,4,2020,BAGAMANOC,MALE,47.0,45 to 49,1,45 to 59,45 to 49


In [13]:
pob_towns['AgeGroup'].unique()

array(['20 to 24', '25 to 29', '45 to 49', '15 to 19', '35 to 39',
       '10 to 14', '40 to 44', '0 to 4', '50 to 54', '30 to 34',
       '55 to 59', '60 to 64', '5 to 9', '75 to 79', '65 to 69',
       '70 to 74', '80+'], dtype=object)

In [15]:
dat_hosp = df.copy()
dat_hosp.head()

,Unnamed: 0,CaseCode,Age,AgeGroup,Sex,DateSpecimen,DateResultRelease,DateRepConf,DateDied,DateRecover,...,ProvRes,CityMunRes,CityMuniPSGC,BarangayRes,BarangayPSGC,HealthStatus,Quarantined,DateOnset,Pregnanttab,ValidationStatus
0,19761,C781369,50.0,50 to 54,MALE,2020-05-30,NaN,2020-06-06,NaN,2020-07-07,...,CATANDUANES,SAN MIGUEL,PH052009000,NaN,NaN,RECOVERED,YES,NaN,NaN,Age or Birthdate is Invalid\r\nCase has Admitt...
1,29313,C636837,47.0,45 to 49,MALE,NaN,NaN,2020-06-22,NaN,2020-07-08,...,CATANDUANES,BAGAMANOC,PH052001000,NaN,NaN,RECOVERED,YES,NaN,NaN,Age or Birthdate is Invalid
2,43311,C951557,1.0,0 to 4,MALE,2020-07-02,2020-07-04,2020-07-06,NaN,2020-07-23,...,CATANDUANES,VIRAC (CAPITAL),PH052011000,NaN,NaN,RECOVERED,NO,2020-06-28,NaN,Age or Birthdate is Invalid
3,45859,C639154,21.0,20 to 24,MALE,2020-06-30,2020-07-05,2020-07-08,NaN,NaN,...,CATANDUANES,CARAMORAN,PH052004000,NaN,NaN,RECOVERED,NO,2020-06-30,NaN,"Removal Type is ""Recovered"", but no Recovered ..."
4,49871,C619176,22.0,20 to 24,FEMALE,2020-07-06,2020-07-07,2020-07-10,NaN,2020-07-23,...,CATANDUANES,VIRAC (CAPITAL),PH052011000,NaN,NaN,RECOVERED,YES,NaN,NO,Age or Birthdate is Invalid


In [16]:
dat_hosp.columns

Index(['Unnamed: 0', 'CaseCode', 'Age', 'AgeGroup', 'Sex', 'DateSpecimen',
       'DateResultRelease', 'DateRepConf', 'DateDied', 'DateRecover',
       'RemovalType', 'DateRepRem', 'Admitted', 'RegionRes', 'ProvRes',
       'CityMunRes', 'CityMuniPSGC', 'BarangayRes', 'BarangayPSGC',
       'HealthStatus', 'Quarantined', 'DateOnset', 'Pregnanttab',
       'ValidationStatus'],
      dtype='object')

In [18]:
dat_hosp = dat_hosp[['DateRepConf', 'Age', 'Sex', 'CityMunRes', 'CityMuniPSGC']]
dat_hosp['AgeGroup'] = dat_hosp['Age'].apply(hosp_func)
dat_hosp = dat_hosp.groupby(['DateRepConf', 'AgeGroup','CityMunRes', 'CityMuniPSGC','Sex']).size().reset_index(name='count')

In [19]:
dat_hosp.head()

,DateRepConf,AgeGroup,CityMunRes,CityMuniPSGC,Sex,count
0,03/17/2023,25 to 29,VIRAC (CAPITAL),PH052011000,MALE,1
1,03/22/2023,25 to 29,VIRAC (CAPITAL),PH052011000,FEMALE,1
2,03/31/2023,55 to 59,VIRAC (CAPITAL),PH052011000,MALE,1
3,04/02/2023,55 to 59,BARAS,PH052002000,MALE,1
4,04/07/2023,65 to 69,VIRAC (CAPITAL),PH052011000,MALE,1


In [20]:
dat_hosp.to_csv('./Data - Catanduanes/1_Processed/dat_hosp.csv')